In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel

param_file = 'Snow17_NN_params.csv'
SWE_sim_file = 'Snow17_NN_SWE.csv'
forcing_file = 'Snow17_NN_forcing.csv'

date_beg_train = '2012-10-01' # This is the first day of water year 2010
date_end_train = '2015-09-30' # This is the last day of water year 2015

date_beg_test = '2015-10-01' # This is the first day of water year 2016
date_end_test = '2020-09-30' # This is the last day of water year 2020

Nens_train = 10


## 2. Split Training and Testing Dataset

In [2]:
# Model parameters
df_params = pd.read_csv(param_file)

# Ensemble SWE simulation
df_swe_sim = pd.read_csv(SWE_sim_file)
df_swe_sim['Date'] = pd.to_datetime(df_swe_sim['Date'],format='%Y-%m-%d')
df_swe_sim.index = df_swe_sim['Date']

df_swe_sim_train = df_swe_sim[date_beg_train:date_end_train]
df_swe_sim_test = df_swe_sim[date_beg_test:date_end_test]

Nt_train = df_swe_sim_train.shape[0]
Nt_test = df_swe_sim_test.shape[0]

## Forcing
df_forcing = pd.read_csv(forcing_file)
df_forcing['Date'] = pd.to_datetime(df_forcing['Date'],format='%Y-%m-%d')
df_forcing.index = df_forcing['Date']

df_forcing_train = df_forcing[date_beg_train:date_end_train]
df_forcing_test = df_forcing[date_beg_test:date_end_test]

Nreps = Nens_train

print('Nt_train = '+str(Nt_train))
print('Nt_test = '+str(Nt_test))
print('Nreps = '+str(Nreps))


Nt_train = 1826
Nt_test = 1827
Nreps = 50


## 3. Assemble Training and Test Datasets Into Correct Format

In [4]:
# Process our "Obervations" we want the neural net to predict. In this case SWE
# Get the SWE ensemble and reshape it into a 1-D array, column-wise 

## TRAINING DATA
SWE_ens_train = df_swe_sim_train.iloc[:,2:(Nreps+2)].values
SWE_ens_1D_train = SWE_ens_train.T.reshape(Nt_train*Nreps,1)

print('Shape of SWE ensemble simulation - training: '+str(SWE_ens_1D_train.shape))

## TESTING DATA
SWE_ens_test = df_swe_sim_test.iloc[:,2:(Nreps+2)].values
SWE_ens_1D_test = SWE_ens_test.T.reshape(Nt_test*Nreps,1)

print('Shape of SWE ensemble simulation - test:'+str(SWE_ens_1D_test.shape))


Shape of SWE ensemble simulation - training: (91300, 1)
Shape of SWE ensemble simulation - test:(91350, 1)


In [5]:
# Process our forcings, making a copy for each ensemble member
## TRAINING DATA
tair_train = df_forcing_train['tair'].values.reshape((Nt_train,1))
tair_ens_train = np.tile(tair_train,(Nreps,1))
print('Shape of tair_ens_train = '+str(tair_ens_train.shape))

pcp_train = df_forcing_train['pcp'].values.reshape((Nt_train,1))
pcp_ens_train = np.tile(pcp_train,(Nreps,1))
print('Shape of pcp_ens_train = '+str(pcp_ens_train.shape))

## TESTING DATA
tair_test = df_forcing_test['tair'].values.reshape((Nt_test,1))
tair_ens_test = np.tile(tair_test,(Nreps,1))
print('Shape of tair_ens_test = '+str(tair_ens_test.shape))

pcp_test = df_forcing_test['pcp'].values.reshape((Nt_test,1))
pcp_ens_test = np.tile(pcp_test,(Nreps,1))
print('Shape of pcp_ens_test = '+str(pcp_ens_test.shape))


Shape of tair_ens_train = (91300, 1)
Shape of pcp_ens_train = (91300, 1)
Shape of tair_ens_test = (91350, 1)
Shape of pcp_ens_test = (91350, 1)


In [7]:
# Process our parameters making a copy for each date
## TRAINING DATA
Dd = df_params['Dd_ens'].iloc[0:Nreps].values.reshape((1,Nreps))
Dd_ens_train = np.tile(Dd,(Nt_train,1)).T.reshape((Nt_train*Nreps,1))

Tt = df_params['Tt_ens'].iloc[0:Nreps].values.reshape((1,Nreps))
Tt_ens_train = np.tile(Tt,(Nt_train,1)).T.reshape((Nt_train*Nreps,1))

print('Shape of Dd_ens_train = '+str(Dd_ens_train.shape))
print('Shape of Tt_ens_train = '+str(Tt_ens_train.shape))

## TESTING DATA
Dd_ens_test = np.tile(Dd,(Nt_test,1)).T.reshape((Nt_test*Nreps,1))
Tt_ens_test = np.tile(Tt,(Nt_test,1)).T.reshape((Nt_test*Nreps,1))

print('Shape of Dd_ens_test = '+str(Dd_ens_test.shape))
print('Shape of Tt_ens_test = '+str(Tt_ens_test.shape))


Shape of Dd_ens_train = (91300, 1)
Shape of Tt_ens_train = (91300, 1)
Shape of Dd_ens_test = (91350, 1)
Shape of Tt_ens_test = (91350, 1)


In [8]:
# Assemble predictor array - training data
SWE_predictors_train = np.concatenate((pcp_ens_train,tair_ens_train,Dd_ens_train,Tt_ens_train),axis=1)
print('SWE_predictors_train dimensions = '+str(SWE_predictors_train.shape))

# Assemble predictor array - test data
SWE_predictors_test = np.concatenate((pcp_ens_test,tair_ens_test,Dd_ens_test,Tt_ens_test),axis=1)
print('SWE_predictors_test dimensions = '+str(SWE_predictors_test.shape))



SWE_predictors_train dimensions = (91300, 4)
SWE_predictors_test dimensions = (91350, 4)


## 4. Train the Neural Network Model

In [9]:
%%time 

kernel = C(1.0, (1e-4, 1e1)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=1)

# Initialize scalers
scaler_predictors = MinMaxScaler()
scaler_target = MinMaxScaler()

# Scale the training and testing predictor data
SWE_predictors_train_scaled = scaler_predictors.fit_transform(SWE_predictors_train)
SWE_predictors_test_scaled = scaler_predictors.transform(SWE_predictors_test)

# Scale the target variable (SWE)
SWE_ens_1D_train_scaled = scaler_target.fit_transform(SWE_ens_1D_train)
SWE_ens_1D_test_scaled = scaler_target.transform(SWE_ens_1D_train)

# clf = MLPRegressor(hidden_layer_sizes=(10,10),
#                    activation='relu',
#                    learning_rate_init=0.001,
#                    random_state=1,
#                    max_iter=2000,
#                    solver='adam').fit(SWE_predictors_train_scaled, SWE_ens_train_scaled.ravel())

gpr = GaussianProcessRegressor(kernel=kernel,
                               n_restarts_optimizer=10,
                               normalize_y=True)

# Fit the GPR model with the scaled data
gpr.fit(SWE_predictors_train_scaled, SWE_ens_1D_train_scaled)



: 

In [ ]:
# Make predictions
swe_ens_1D_predict_test_scaled = clf.predict(SWE_predictors_test_scaled)

# Inverse transform the predictions to original scale
swe_ens_1D_predict_test = scaler_target.inverse_transform(swe_ens_1D_predict_test_scaled.reshape(-1, 1))

#swe_ens_1D_predict_test = clf.predict(SWE_predictors_test)
print(swe_ens_1D_predict_test.shape)

In [ ]:
swe_ens_predict_test = swe_ens_1D_predict_test.reshape((Nt_test,Nreps))
print(swe_ens_predict_test.shape)

In [ ]:
ens = 1

plt.figure(figsize=(8,10))
plt.plot(df_forcing_test['Date'],swe_ens_predict_test[:,ens],label=['Emulator, ensemble member'+str(ens)])
plt.plot(df_forcing_test['Date'],SWE_ens_test[:,ens],label=['Snow 17, ensemble member'+str(ens)])
plt.xlabel('Date')
plt.ylabel('SWE [mm]')
plt.show()

In [85]:
swe_ens_1D_predict_train_scaled = clf.predict(SWE_predictors_train_scaled)

swe_ens_1D_predict_train = scaler_target.inverse_transform(swe_ens_1D_predict_train_scaled.reshape(-1,1))

swe_ens_predict_train = swe_ens_1D_predict_train.reshape((Nt_train,Nreps))

In [ ]:
plt.plot(swe_ens_predict_train[:,200])

